##Scrape IMDb to get the list of nominations and list of awards won by each movie. ##
This will be our results.

In [10]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import re
import string
import json
from tqdm import tqdm

###Scrape IMDb for nomination information###

In [11]:
df = pd.read_pickle('wiki_df_cleaned.csv')

Check how many movies there are in total.

In [19]:
df.head()
df.shape[0]

1275

After scraping, we have 1265 movies, but that shouldn't be a problem, because we will delete the moview with run time less than 1 hour.

In [13]:
movietext={}
for m_id in tqdm(df.imdb_id,leave=True):
    pg="http://www.imdb.com/title/{}/awards?ref_=tt_awd".format(m_id)
    movietext[m_id]=requests.get(pg)
    

len(movietext)

100%|██████████| 1275/1275 [07:25<00:00,  3.96it/s]


1265

In [129]:
##test
m_id=movietext.keys()[20]
#test
m_id='tt0018389'
print m_id
movietextdict = movietext[m_id]
soup = BeautifulSoup(movietextdict.text, "html.parser")
rows = soup.find("table", attrs={"class":"awards"}).find_all("tr")[0:]
nominations=[]
wins=[]
won_q= "Won" in rows[0].find("td", attrs={"class":"title_award_outcome"}).get_text()
#print "Nominated" in rows[5].find("td", attrs={"class":"title_award_outcome"})


for i,r in enumerate(rows):
    #print i
    if r.find("td", attrs={"class":"title_award_outcome"})!=None:
        check= "Nominated" in r.find("td", attrs={"class":"title_award_outcome"}).get_text()
        if check==True:
            #print "t",i
            nn=i
        
#print nn

for i,r in enumerate(rows):
    print r
    rr=r.find_all("td", attrs={"class":"award_description"})[0:]
    if i<nn:
        for i in rr:
            if "Best Actor in a Leading Role" in i.get_text():
                wins.append('ML')
            if "Best Picture" in i.get_text():
                wins.append('BP')
            if "Best Actess in a Leading Role" in i.get_text():
                wins.append('FL')
            if "Best Actess in a Supporting Role" in i.get_text():
                wins.append('FS')
            if "Best Actor in a Supporting Role" in i.get_text():
                wins.append('MS')  
    else:
        for i in rr:
            if "Best Actor in a Leading Role" in i.get_text():
                nominations.append('ML')
            if "Best Picture" in i.get_text():
                nominations.append('BP')
            if "Best Actess in a Leading Role" in i.get_text():
                nominations.append('FL')
            if "Best Actess in a Supporting Role" in i.get_text():
                nominations.append('FS')
            if "Best Actor in a Supporting Role" in i.get_text():
                nominations.append('MS')   
nominations=nominations+wins
print nominations, wins

tt0018389
<tr>
<td class="title_award_outcome" rowspan="1">
<b>Nominated</b><br/>
<span class="award_category">Oscar</span>
</td>
<td class="award_description">
            Best Actress in a Leading Role<br/>
<a href="/name/nm0237571?ref_=ttawd_awd_1">Louise Dresser</a>
<br/>
</td>
</tr>
[] []


A function to get the nominations information. Will return a three things: movie id, list of nominations, list of wins

In [138]:
def get_nomination(m_id,movietext):
    #print m_id
    movietextdict = movietext[m_id]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    nominations=[]
    wins=[]
    if soup.find("table", attrs={"class":"awards"})!=None:
        rows = soup.find("table", attrs={"class":"awards"}).find_all("tr")[0:]
        nn=0
        won_q= "Won" in rows[0].find("td", attrs={"class":"title_award_outcome"}).get_text()
        #print "Nominated" in rows[5].find("td", attrs={"class":"title_award_outcome"})


        for i,r in enumerate(rows):
            #print i
            if r.find("td", attrs={"class":"title_award_outcome"})!=None:
                check= "Nominated" in r.find("td", attrs={"class":"title_award_outcome"}).get_text()
                if check==True:
                    #print "t",i
                    nn=i

        #print nn

        for i,r in enumerate(rows):
            rr=r.find_all("td", attrs={"class":"award_description"})[0:]
            if i<nn:
                for ii in rr:
                    if "Best Actor in a Leading Role" in ii.get_text():
                        wins.append('ML')
                    if "Best Picture" in ii.get_text():
                        wins.append('BP')
                    if "Best Actress in a Leading Role" in ii.get_text():
                        wins.append('FL')
                    if "Best Actress in a Supporting Role" in ii.get_text():
                        wins.append('FS')
                    if "Best Actor in a Supporting Role" in ii.get_text():
                        wins.append('MS')  
            else:
                for ii in rr:
                    if "Best Actor in a Leading Role" in ii.get_text():
                        nominations.append('ML')
                    if "Best Picture" in ii.get_text():
                        nominations.append('BP')
                    if "Best Actress in a Leading Role" in ii.get_text():
                        nominations.append('FL')
                    if "Best Actress in a Supporting Role" in ii.get_text():
                        nominations.append('FS')
                    if "Best Actor in a Supporting Role" in ii.get_text():
                        nominations.append('MS')   
        nominations=nominations+wins
    
    return m_id, nominations, wins

In [139]:
##test
m_id='tt0018389'
get_nomination(m_id,movietext)

('tt0018389', ['FL'], [])

We now merge into dataframe.

In [140]:
bp_w=[]
ms_w=[]
fs_w=[]
ml_w=[]
fl_w=[]
bp_n=[]
ms_n=[]
fs_n=[]
ml_n=[]
fl_n=[]
movie_id=[]
for mid in movietext.keys():
    mid,n,w=get_nomination(mid,movietext)
    movie_id.append(mid)
    
    bp_w.append("BP" in w)
    ms_w.append("MS" in w)
    fs_w.append("FS" in w)
    ml_w.append("ML" in w)
    fl_w.append("FL" in w)
    
    bp_n.append("BP" in n)
    ms_n.append("MS" in n)
    fs_n.append("FS" in n)
    ml_n.append("ML" in n)
    fl_n.append("FL" in n)

print len(movie_id),len(bp_w),len(ms_w),len(fs_w),len(ml_w),len(fl_w),len(bp_n),len(ms_n),len(fs_n),len(ml_n),len(fl_n)
        
    

1265 1265 1265 1265 1265 1265 1265 1265 1265 1265 1265


In [141]:
dict_for_df={}
dict_for_df['bp_w']=bp_w
dict_for_df['ms_w']=ms_w
dict_for_df['fs_w']=fs_w
dict_for_df['fl_w']=fl_w
dict_for_df['ml_w']=ml_w
dict_for_df['bp_n']=bp_n
dict_for_df['ms_n']=ms_n
dict_for_df['fs_n']=fs_n
dict_for_df['ml_n']=ml_n
dict_for_df['fl_n']=fl_n
dict_for_df['movie_id']=movie_id

dict_for_df.keys()

['ms_n',
 'bp_w',
 'fl_n',
 'fs_w',
 'ml_w',
 'ml_n',
 'movie_id',
 'ms_w',
 'fl_w',
 'bp_n',
 'fs_n']

In [142]:
df1=pd.DataFrame(dict_for_df)

In [143]:
df1.head()

,bp_n,bp_w,fl_n,fl_w,fs_n,fs_w,ml_n,ml_w,movie_id,ms_n,ms_w
0,False,False,False,False,False,False,True,False,tt0070666,False,False
1,False,False,False,False,False,False,False,False,tt2582802,False,False
2,False,False,False,False,True,False,False,False,tt0172493,False,False
3,True,False,False,False,False,False,True,False,tt0045943,False,False
4,False,False,True,False,False,False,True,False,tt0055895,False,False


In [144]:
df1.to_pickle('result.csv')